In [1]:
import pandas as pd
import numpy as np
from random import randrange
import math
import json

# READING FILES

In [2]:
# Read lookup for geographic hierarchy
tiers = pd.read_csv('./LAD_County_Region_Country_2021.csv')

# Combine small areas
tiers = tiers.append({'LAD21CD': 'E09000001/E09000033', 'LAD21NM': 'City of London/Westminster', 'CTY21CD': 'E13000001', 'CTY21NM': 'Inner London', 'RGN21CD': 'E12000007', 'RGN21NM': 'London', 'CTRY21CD': 'E92000001', 'CTRY21NM': 'England'}, ignore_index=True)

tiers = tiers.append({'LAD21CD': 'E06000052/E06000053', 'LAD21NM': 'Cornwall/Isles of Scilly', 'CTY21CD': 'E06000052/E06000053', 'CTY21NM': 'Cornwall/Isles of Scilly', 'RGN21CD': 'E12000009', 'RGN21NM': 'South West', 'CTRY21CD': 'E92000001', 'CTRY21NM': 'England'}, ignore_index=True)

# Remove the small areas (uncombined)
tiers = tiers[tiers['LAD21NM'].apply(lambda x: x not in ['Cornwall', 'Isles of Scilly', 'Westminster', 'City of London'])]

tiers

,LAD21CD,LAD21NM,CTY21CD,CTY21NM,RGN21CD,RGN21NM,CTRY21CD,CTRY21NM
0,E06000001,Hartlepool,E06000001,Hartlepool,E12000001,North East,E92000001,England
1,E06000002,Middlesbrough,E06000002,Middlesbrough,E12000001,North East,E92000001,England
2,E06000003,Redcar and Cleveland,E06000003,Redcar and Cleveland,E12000001,North East,E92000001,England
3,E06000004,Stockton-on-Tees,E06000004,Stockton-on-Tees,E12000001,North East,E92000001,England
4,E06000005,Darlington,E06000005,Darlington,E12000001,North East,E92000001,England
...,...,...,...,...,...,...,...,...
328,W06000022,Newport,W06000022,Newport,W92000004,Wales,W92000004,Wales
329,W06000023,Powys,W06000023,Powys,W92000004,Wales,W92000004,Wales
330,W06000024,Merthyr Tydfil,W06000024,Merthyr Tydfil,W92000004,Wales,W92000004,Wales
331,E09000001/E09000033,City of London/Westminster,E13000001,Inner London,E12000007,London,E92000001,England


In [3]:
# Open data files containing nearest neighbour information and process into dictionaries
with open('ladsneighbours.json') as json_file:
    lads_neigh_raw = json.load(json_file)
lads_neigh = {}
for i in lads_neigh_raw:
    if (lads_neigh_raw[i][0]):
        lads_neigh[i] = lads_neigh_raw[i][0][0]
    else:
        lads_neigh[i] = lads_neigh_raw[i][1]

# Add nearest neighbour for combined small areas Cornwall and Isles of Scilly and CoL/Westminster
lads_neigh['E06000052/E06000053'] = lads_neigh['E06000052']
lads_neigh['E09000001/E09000033'] = lads_neigh['E09000033']

countyneighbourdf = pd.read_csv('countyboundaries.csv')
county_neigh = {}
for i in countyneighbourdf.index:
    countycodelist = countyneighbourdf.iloc[i]['neighbours'].split(",")
    countylengthlist = countyneighbourdf.iloc[i]['lengths'].split(",")
    zipped = list(zip(countycodelist, countylengthlist))
    zipped = [i for i in zipped if i[0][:3] != "E08"]
    zipped = sorted(zipped, reverse=True, key= lambda x: int(x[1]))
    county_neigh[countyneighbourdf.iloc[i]['CTYUA21CD']] = zipped[0][0]

# Add nearest neighbours for met counties as they weren't included in boudary data
county_neigh['E11000001'] = 'E10000017'
county_neigh['E11000002'] = 'E10000017'
county_neigh['E11000003'] = 'E10000007'
county_neigh['E11000007'] = 'E06000057'
county_neigh['E11000005'] = 'E10000031'
county_neigh['E11000006'] = 'E10000023'
county_neigh['E13000001'] = 'E13000002'
county_neigh['E13000002'] = 'E13000001'
# Add nearest neighbour for combined Cornwall and Isles of Scilly
county_neigh['E06000052/E06000053'] = county_neigh['E06000052']

In [4]:
# Open and process data on statistical similarity
similars = {}
similar_raw = pd.read_csv('./LAD_MAY_2021_EW_Adjacency_Matrix.csv')
for i in similar_raw.index:
    similars[similar_raw.iloc[i]['LAD21CD']] = [similar_raw.iloc[i]['LAD1CD'],similar_raw.iloc[i]['LAD2CD'],similar_raw.iloc[i]['LAD3CD'],similar_raw.iloc[i]['LAD4CD']]

similars['E06000052/E06000053'] = similars['E06000052']
similars['E09000001/E09000033'] = similars['E09000033']

# CREATE THE EMPTY OBJECTS FOR EACH PLACE

In [5]:
# A function to find a statistically similar area
def get_similar(code):
    # Create a list of similar areas
    theseSimilars = similars[code]
    # Get rid of NaNs for the list of similars
    theseSimilars = [i for i in theseSimilars if i == i]

    # Filter out codes for Scotland and NI
    theseSimilars = [i for i in theseSimilars if i[0] not in ['S', 'N']]

    # Remove small areas
    theseSimilars = [i for i in theseSimilars if i not in ['E09000001', 'E09000033', 'E06000053', 'E06000052']]

    # Create variable for most similar area
    if len(theseSimilars) > 0:
        similar = theseSimilars[0]
    else:
        similar = None

    # If the most similar area is also the nearest area, then chose another area
    if similar == las[nameLA]['near']:
        # If there's at least two similar areas reselect area
        if len(theseSimilars)>1:
            similar = theseSimilars[1]
        else:
            similar = None

    return similar

In [6]:
# Create and object to populate with each area's data
las = {}
counties = {}
regions = {}
countries = {}

# Iterate throough index of LA df
for i in tiers.index:

    # Create a variable which is an iterated row of the df
    row = tiers.loc[i]

    # The name of the area attached to this row
    nameLA = row['LAD21NM']
    nameCounty = row['CTY21NM']
    nameRegion = row['RGN21NM']
    nameCountry = row['CTRY21NM']

    # Add an object with name and code info and empty nested oobject for data
    las[nameLA] = {}
    las[nameLA]['name'] = nameLA
    las[nameLA]['parent'] = nameRegion
    las[nameLA]['country'] = nameCountry
    las[nameLA]['code'] = row['LAD21CD']
    las[nameLA]['type'] = 'LAD'
    las[nameLA]['data'] = {}

    counties[nameCounty] = {}
    counties[nameCounty]['name'] = nameCounty
    counties[nameCounty]['parent'] = nameRegion
    counties[nameCounty]['country'] = nameCountry
    counties[nameCounty]['code'] = row['CTY21CD']
    counties[nameCounty]['near'] = county_neigh[row['CTY21CD']]
    counties[nameCounty]['type'] = 'County'
    counties[nameCounty]['data'] = {}

    regions[nameRegion] = {}
    regions[nameRegion]['name'] = nameRegion
    regions[nameRegion]['parent'] = nameCountry
    regions[nameRegion]['country'] = nameCountry
    regions[nameRegion]['code'] = row['CTY21CD']
    regions[nameRegion]['type'] = 'Region'
    regions[nameRegion]['data'] = {}

    countries[nameCountry] = {}
    countries[nameCountry]['name'] = nameCountry
    countries[nameCountry]['code'] = row['CTRY21CD']
    countries[nameCountry]['type'] = 'Country'
    countries[nameCountry]['data'] = {}

# DATA PROCESSING BEGINS BELOW

In [7]:
# LIST OF ALL TOPICS AND BOTH YEARS
topics = ['care',
 'religion',
 'ethnicity',
 'health',
 'economic',
 'household',
 'marital',
 'hours',
 'tenure',
 'disability',
 'national',
 'welsh',
 'population',
 'density',
 'age10yr',
 'agemed']
years = ['2011', '2021']

In [8]:
def merge_small(df):
    cityolondon = df[df['Area Name'] == 'City of London']
    westmin = df[df['Area Name'] == 'Westminster']

    cityolondon = cityolondon.drop(['Area Code', 'Area Name'], axis=1)
    westmin = westmin.drop(['Area Code', 'Area Name'], axis=1)

    cityolondon = cityolondon.reset_index(drop=True)
    westmin = westmin.reset_index(drop=True)

    colandwest = cityolondon.add(westmin, fill_value=0)
    colandwest['Area Code'] = 'E09000001/E09000033'
    colandwest['Area Name'] = 'City of London/Westminster'

    df = df.append(colandwest)

    cornwall = df[df['Area Name'] == 'Cornwall']
    scilly = df[df['Area Name'] == 'Isles of Scilly']

    cornwall = cornwall.drop(['Area Code', 'Area Name'], axis=1)
    scilly = scilly.drop(['Area Code', 'Area Name'], axis=1)

    cornwall = cornwall.reset_index(drop=True)
    scilly = scilly.reset_index(drop=True)

    cornandscilly = cornwall.add(scilly, fill_value=0)
    cornandscilly['Area Code'] = 'E06000052/E06000053'
    cornandscilly['Area Name'] = 'Cornwall/Isles of Scilly'

    df = df.append(cornandscilly)

    df = df[df['Area Name'].apply(lambda x: x not in ['Westminster', 'City of London', 'Cornwall', 'Isles of Scilly'])]
    df = df.reset_index(drop=True)

    return df

### Adding values

In [9]:
# Create an object that will hold all the dfs for all las in an indexable way
data_las = {}
data_counties = {}
data_regions = {}
data_countries = {}

for topic in topics:
    for year in years:

        # Read a csv
        df = pd.read_csv("./"+topic+"_"+year+".csv")

        # For some reason some areas have an astrix in the title
        # df['Area Name'] = df['Area Name'].str.replace("*", "")
        df['Area Name'] = df['Area Name'].str.replace("\(Met County\)", "").str.replace("()", "").str.strip() 

        # Merge data for small areas
        df = merge_small(df)

        # Create a df including only LAs
        data_las[topic+"_"+year] = df[df['Area Name'].apply(lambda x: x in las.keys())]

        # Create a df including only counties
        data_counties[topic+"_"+year] = df[df['Area Name'].apply(lambda x: x in counties.keys())]

        # Reformat Captialised textfor regions and countries
        df['Area Name'] = df['Area Name'].apply(lambda x: x.title().replace(" And ", " and ").replace(" Of ", " of "))

        # Create df for regions
        data_regions[topic+"_"+year] = df[df['Area Name'].apply(lambda x: x in regions.keys())]

        # Create df for regions
        data_countries[topic+"_"+year] = df[df['Area Name'].apply(lambda x: x in countries.keys())]

In [10]:
# These variables hold the relevant data tables and the objects that will be populated with the data, indexible by 
geog_tables = {'las': data_las, 'counties': data_counties, 'regions': data_regions, 'countries': data_countries}
geog_objects = {'las': las, 'counties': counties, 'regions': regions, 'countries': countries}

In [11]:
def find_data(name, geog):

    data = {}

    for topic in topics:

        # Create object for this topic
        data[topic] = {}
        data[topic]['value'] = {}
        data[topic]['perc'] = {}

        for year in years:

            # Find table corresponding to topic and year
            table = geog_tables[geog][topic+'_'+year]

            row = table[table['Area Name'] == name].iloc[0]

            # Find the row of the table relating to the area of iteration
            row = table[table['Area Name'] == name].iloc[0]

            # Transform row into dictionary
            row = row.to_dict()

            # Delete unneeded values
            del row['Area Code']
            del row['Area Name']

            # Assign to main data object
            data[topic]['value'][year] = row

            # Calculate percentages: this calcualtion assumes that each figure combines to make a total
            perc = {i: round(row[i]/sum(row.values()), 5) for i in row}
            data[topic]['perc'][year] = perc

        for value_perc in ['value', 'perc']:
            # Calculate percentage point changes
            ob2 = data[topic][value_perc]['2021']
            ob1 = data[topic][value_perc]['2011']
            percob = {i: round(ob2[i] - ob1[i], 5) for i in ob2}
            data[topic][value_perc]['change'] = percob

    return data

In [12]:
# ADDING VALUE DATA FOR EACH TOPIC AND YEAR

for name in las.keys():
    las[name]['data'] = find_data(name, 'las')

for name in counties.keys():
    counties[name]['data'] = find_data(name, 'counties')

for name in regions.keys():
     regions[name]['data'] = find_data(name, 'regions')

for name in countries.keys():
     countries[name]['data'] = find_data(name, 'countries')

### Adding ranks

In [13]:
# This function takes in a raw index and outputs a rank
def process_rank(raw_rank, group_size):
    half_group = round(group_size/2)
    rank = raw_rank+1

    # If ranked in the bottom half of the group the number is given a negative number. So the last will be -1, second last -2
    if rank > half_group:
        rank = (1+(group_size-rank))*-1

    return rank

def create_ranks(name, topic, geog, hier, value_perc):
    places = geog_objects[geog]
    ob = {}
    for type in ['2011', '2021', 'change']:
        ob[type] = {}
        for var in places[name]['data'][topic][value_perc][type].keys():

            # Find the value of the data that is being ranked
            this_data = places[name]['data'][topic][value_perc][type][var]

            # Create list of values. If regional rank then filter by areas that share the same parent or country
            if hier == 'regional':
                group_data = [places[i]['data'][topic][value_perc][type][var] for i in places if places[i]['parent'] == places[name]['parent']]
            else:
                group_data = [places[i]['data'][topic][value_perc][type][var] for i in places if places[i]['country'] == places[name]['country']]
            group_size = len(group_data)

            # Sort the data then find the index of the data point we are ranking
            group_data.sort()
            raw_rank = group_data.index(this_data)

            # Process the index into a rank and add to the object that will be outputted
            ob[type][var] = process_rank(raw_rank, group_size)
            
    return ob

In [14]:
# ADDING PERCENTAGE DATA FOR EACH TOPIC AND YEAR - NOT NEEDED FOR REGIONS AND COUNTRIES

for name in las.keys():
    for topic in topics:
        las[name]['data'][topic]['perc_rank'] = create_ranks(name, topic, 'las', 'national', 'perc')
        las[name]['data'][topic]['perc_rank_local'] = create_ranks(name, topic, 'las', 'regional', 'perc')
        las[name]['data'][topic]['value_rank'] = create_ranks(name, topic, 'las', 'national', 'value')
        las[name]['data'][topic]['value_rank_local'] = create_ranks(name, topic, 'las', 'regional', 'value')


for name in counties.keys():
    for topic in topics:
        counties[name]['data'][topic]['perc_rank'] = create_ranks(name, topic, 'counties', 'national', 'perc')
        counties[name]['data'][topic]['perc_rank_local'] = create_ranks(name, topic, 'counties', 'regional', 'perc')
        counties[name]['data'][topic]['value_rank'] = create_ranks(name, topic, 'counties', 'national', 'value')
        counties[name]['data'][topic]['value_rank_local'] = create_ranks(name, topic, 'counties', 'regional', 'value')

#### Find each LA and county's near and similar areas

In [15]:
# This is done after first iteration to point to other objects in dictionary
for i in tiers.index:

    # Create a variable which is an iterated row of the df
    row = tiers.loc[i]

    # The name of the area attached to this row
    nameLA = row['LAD21NM']
    nameCounty = row['CTY21NM']

    # print(row['LAD21NM'])
    # print(row['LAD21CD'])

    # las[nameLA]['near'] = lads_neigh[row['LAD21CD']]
    las[nameLA]['near'] = [las[i] for i in las if las[i]['code'] == lads_neigh[row['LAD21CD']]][0].copy()
    # las[nameLA]['near']['near'] = 'confudsed'
    las[nameLA]['near']['near'] = ""
    del las[nameLA]['near']['near']
    las[nameLA]['near']['similar'] = ""
    del las[nameLA]['near']['similar']

    # las[nameLA]['similar'] = get_similar(row['LAD21CD'])
    if get_similar(row['LAD21CD']):
        las[nameLA]['similar'] = [las[i] for i in las if las[i]['code'] == get_similar(row['LAD21CD'])][0].copy()
        las[nameLA]['similar']['near'] = ""
        del las[nameLA]['similar']['near']
        las[nameLA]['similar']['similar'] = ""
        del las[nameLA]['similar']['similar']

    # counties[nameCounty]['near'] = county_neigh[row['CTY21CD']] 
    counties[nameCounty]['near'] = [counties[i] for i in counties if counties[i]['code'] == county_neigh[row['CTY21CD']]][0].copy()
    counties[nameCounty]['near']['near'] = ""
    del counties[nameCounty]['near']['near']
    counties[nameCounty]['near']['similar'] = ""
    del counties[nameCounty]['near']['similar']

# Below is the story finding section

In [16]:
# Add a stories array for each area
for name in las:
    place = las[name]
    stories = []
    for a in place['data'].keys():
        for d in place['data'][a]['perc']['change'].keys():
            if a in ['population', 'agemed']:
                b = 'value'
            else:
                b = 'perc'
            stories.append({
                'label': a+'_'+b+'_change'+'_'+d, 
                'locRank': place['data'][a][b+'_rank_local']['change'][d], 
                'natRank': place['data'][a][b+'_rank']['change'][d], 
                '2011': place['data'][a][b]['2011'][d],
                '2021': place['data'][a][b]['2021'][d],
                'change': place['data'][a][b]['change'][d],
                'perc_cha': 100*(place['data'][a][b]['change'][d]/place['data'][a][b]['2011'][d])
            })
    place['stories'] = stories

In [17]:
# A function which outputs 1.0 for any positive number and -1.0 for any negative number
sign = lambda x: math.copysign(1, x)
sign(-9.7)

# A function which takes in a list of stories and combines the 'type' lists of any repeated variables and gets rid of the duplcate
def combinestories(stories):
    stories2 = []
    for s in stories:
        if s['label'] in [i['label'] for i in stories2]:
            index = next((i for i, item in enumerate(stories2) if item['label'] == s['label']), -1)
            stories2[index]['type'] = stories2[index]['type'] + s['type']
        else:
            stories2.append(s)

    return stories2

# Functions that find regional and national equivelent to a local data point
def reg(story):
    s=story['label'].split("_")
    if len(s)>4:
        s[3] = s[3] + "_" + s[4]
    return region['data'][s[0]][s[1]][s[2]][s[3]]

def cou(i):
    s=i['label'].split("_")
    if len(s)>4:
        s[3] = s[3] + "_" + s[4]
    return country['data'][s[0]][s[1]][s[2]][s[3]]

def simi(i):
    s=i['label'].split("_")
    if len(s)>4:
        s[3] = s[3] + "_" + s[4]
    return similar['data'][s[0]][s[1]][s[2]][s[3]]

def near(i):
    s=i['label'].split("_")
    if len(s)>4:
        s[3] = s[3] + "_" + s[4]
    return nearby['data'][s[0]][s[1]][s[2]][s[3]]

In [18]:
for name in las:
    place = las[name]
    region = regions[place['parent']]
    country = countries[place['country']]
    # Skip similar-area based stories if this LA doesn't have a similar area
    if 'similar' in place:
        similar = place['similar']
    nearby = place['near']

    # Create a list of objects ordered by the size of difference between regional and local change
    regDiff = [(i, {'otherchange': reg(i), 'dif': i['change']-reg(i)}) for i in place['stories']]
    couDiff = [(i, {'otherchange': cou(i), 'dif': i['change']-cou(i)}) for i in place['stories']]
    if 'similar' in place:
        simiDiff = [(i, {'otherchange': simi(i), 'dif': i['change']-simi(i)}) for i in place['stories']]
    nearDiff = [(i, {'otherchange': near(i), 'dif': i['change']-near(i)}) for i in place['stories']]

    # Create list of objects ordered by size of PP change
    vallist = sorted(place['stories'], reverse=True, key=lambda x: x['change'])

    # Iterate through vallist to find the largest percentage point increase for ethnicity and religion
    ethStory = next(filter(lambda x: x['label'].split("_")[0] == 'ethnicity', vallist))
    relStory = next(filter(lambda x: x['label'].split("_")[0] == 'religion', vallist))

    # Add the story type descriptor for these two stories
    ethStory = [{**ethStory, **{'type':['ethrel']}}]
    relStory = [{**relStory, **{'type':['ethrel']}}]

    # Remove all religion and ethnicity variables from our original story list and add the two stories pre-selected
    place['stories'] = [i for i in place['stories'] if (i['label'].split("_")[0]!='religion') & (i['label'].split("_")[0]!='ethnicity')]+ethStory+relStory

    # This records the story type for each variable in our stories list
    stories = []
    if (country['name']=='Wales'):
        stories=stories+[{**i, **{'type':['welsh']}} for i in place['stories'] if i['label'] == 'welsh_perc_change_Speaks Welsh']
    stories+[{**i, **{'type':['pop']}} for i in place['stories'] if i['label'] == 'population_value_change_Population']
    stories=stories+[{**i, **{'type':['size']}} for i in place['stories'] if ((abs(i['change'])/abs(i['2011'])>0.2)&(abs(i['change']) > 1))]
    stories=stories+[{**i, **{'type':['locRank']}} for i in place['stories'] if (abs(i['locRank']) < 4)&(abs(i['change']) > 0.5)]
    stories=stories+[{**i, **{'type':['natRank']}} for i in place['stories'] if (abs(i['natRank']) < 4)&(abs(i['change']) > 0.5)]
    stories=stories+[{**i[0], **{'type':['couBuck']}} for i in couDiff if ((abs(i[1]['otherchange'])>1) & (abs(i[0]['change'])>1) & (sign(i[1]['otherchange'])!=sign(i[0]['change'])))]
    stories=stories+[{**i[0], **{'type':['regBuck']}} for i in regDiff if ((abs(i[1]['otherchange'])>1) & (abs(i[0]['change'])>1) & (sign(i[1]['otherchange'])!=sign(i[0]['change'])))]
    stories=stories+[{**i[0], **{'type':['couDiff']}} for i in couDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['otherchange'])==sign(i[0]['change'])))]
    stories=stories+[{**i[0], **{'type':['regDiff']}} for i in regDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['otherchange'])==sign(i[0]['change'])))]
    stories=stories+[{**i[0], **{'type':['nearDiff']}} for i in nearDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['otherchange'])==sign(i[0]['change'])))]
    if 'similar' in place:
        stories=stories+[{**i[0], **{'type':['simiDiff']}} for i in simiDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['otherchange'])==sign(i[0]['change'])))]
    stories=stories+[{**i, **{'type':['general']}} for i in place['stories']]
    stories = combinestories(stories)

    # This section puts stories in order based on how notable each variable is
    stories = sorted(stories, reverse=True, key=lambda x: abs(x['locRank'])<4)
    stories = sorted(stories, reverse=True, key=lambda x: abs(x['natRank'])<4)
    stories = sorted(stories, reverse=True, key=lambda x: abs(x['locRank'])<3)
    stories = sorted(stories, reverse=True, key=lambda x: abs(x['natRank'])<3)
    stories = sorted(stories, reverse=True, key=lambda x: x['type']=='regBuck')
    stories = sorted(stories, reverse=True, key=lambda x: x['type']=='couBuck')
    stories = sorted(stories, reverse=True, key=lambda x: x['label']=='welsh_perc_change_Speaks Welsh')
    stories = sorted(stories, reverse=True, key=lambda x: x['label']=='agemed_value_change_Median Age')
    stories = sorted(stories, reverse=True, key=lambda x: x['label']=='population_value_change_Population')

    # Remove stories from topics that have appeared higher up in the story list
    stories_uniquetopic = []
    dontInc = ['density', 'age10yr']
    for story in stories:
        if story['label'].split("_")[0] not in dontInc:
            stories_uniquetopic.append(story)
            dontInc.append(story['label'].split("_")[0])

    place['stories'] = stories_uniquetopic

In [19]:
place['stories']

[{'label': 'population_value_change_Population',
  'locRank': -5,
  'natRank': -33,
  '2011': 80679,
  '2021': 111350,
  'change': 30671,
  'perc_cha': 38.01608844928668,
  'type': ['size', 'couBuck', 'regDiff', 'general']},
 {'label': 'agemed_value_change_Median Age',
  'locRank': -11,
  'natRank': -116,
  '2011': 15049,
  '2021': 21442,
  'change': 6393,
  'perc_cha': 42.481227988570666,
  'type': ['size', 'couDiff', 'regDiff', 'nearDiff', 'simiDiff', 'general']},
 {'label': 'welsh_perc_change_Speaks Welsh',
  'locRank': -13,
  'natRank': -121,
  '2011': 0.53711,
  '2021': 0.55892,
  'change': 0.02181,
  'perc_cha': 4.060620729459515,
  'type': ['general']},
 {'label': 'economic_perc_change_Economically inactive: Looking after home or family',
  'locRank': 2,
  'natRank': 12,
  '2011': 0.11969,
  '2021': 0.0746,
  'change': -0.04509,
  'perc_cha': -37.6723201604144,
  'type': ['general']},
 {'label': 'household_perc_change_Single family household: Married or civil partnership couple:

In [ ]:
# # Create an empty array where selected stories will be passed
# newStories = []

# # Iterate through each area's list of stories
# for story in place['stories']:

#     # Create array from story label and format last point incase variable includes '_'
#     s=story['label'].split("_")
#     if len(s)>4:
#         s[3] = s[3] + "_" + s[4]

#     # Don't add Welsh story for non-Welsh areas
#     if (s[0]=='welsh') & (place['parent'] != 'Wales'):
#         pass

#     else:
#         newStories.append(story)


In [ ]:
# # Filter out priority list by subject
# for lad in las:
#     newStories = []
#     for story in lad['stories']:
#         s=story['label'].split("_")
#         if len(s)>4:
#             s[3] = s[3] + "_" + s[4]
#         if (s[0]=='welsh') & (lad['parents'][0]['name'] != 'Wales'):
#             pass
#         else:
#             story['2011'] = [i for i in lad['Priorities'] if i['label']==s[0]+'_'+s[1]+'_2011_'+s[3]][0]['value']
#             story['perccha'] = round(100*((story['value']+0.001)/([i for i in lad['Priorities'] if i['label']==s[0]+'_'+s[1]+'_2001_'+s[3]][0]['value']+0.001)),2)
#             newStories.append(story)
#         if (s[2]!="change"):
#             newStories.append(story)
#     lad['stories'] = newStories